In [5]:
import os
import pyproj
import rasterio
from rasterio.mask import mask
from shapely.geometry import Polygon, mapping

In [6]:
def getCutPoint(filePath):
    ''' 
    open cutpoint txt file
    the set the list to a,b,d,c for create shapefile
    '''
    returnList = []
    coorList = []
    with open(filePath, "r") as file:
        lines = file.readlines()

    pointCount = 0
    eachShapePoint = []
    for eachPoint in lines:
        eachPoint = eachPoint.strip()
        if not eachPoint:
            continue
        latlon = eachPoint.split(",")
        eachShapePoint.append((float(latlon[1]), float(latlon[0])))
        pointCount += 1

        if pointCount == 4:
            coorList.append(eachShapePoint)
            pointCount = 0 
            eachShapePoint = []

    for a, b, c, d in coorList:
        returnList.append([a, b, d, c])

    return returnList


def getRGBTifFile(filePath):
    returnList = []
    for tifFile in os.listdir(filePath):
        tifFilePath = os.path.join(filePath, tifFile)
        returnList.append(tifFilePath)

    return returnList

def lat_lon_to_epsg32654(latitude, longitude):
    """
    Converts latitude and longitude coordinates to EPSG:32654 (UTM Zone 54N).

    Args:
        latitude: Latitude in decimal degrees.
        longitude: Longitude in decimal degrees.

    Returns:
        Tuple containing the easting and northing coordinates in EPSG:32654.
    """
    try:
        # Define coordinate systems
        wgs84 = pyproj.CRS("EPSG:4326")  # WGS 84 (latitude/longitude)
        utm54n = pyproj.CRS("EPSG:32654") 

        # Create a transformation object
        transformer = pyproj.Transformer.from_crs(wgs84, utm54n)

        # Perform the coordinate transformation
        easting, northing = transformer.transform(longitude, latitude) 

        return easting, northing

    except pyproj.exceptions.CRSError:
        print("Error during coordinate transformation.")
        return None
    
def cutPix4dRGBUsingCutPoints(inputTifFilePath, outputPath, coords):
    """
    Cut a .tif file using 4 coordinates (lat, lon) and save cropped output.
    """
    PreprocessedCoor = []
    for eachCoor in coords:
        easting, northing = lat_lon_to_epsg32654(eachCoor[0], eachCoor[1])
        PreprocessedCoor.append((easting, northing))

    polygon_coords = PreprocessedCoor.copy()
    polygon_coords.append(polygon_coords[0])  # close polygon

    polygon = Polygon(polygon_coords)
    geojson_geom = [mapping(polygon)]

    with rasterio.open(inputTifFilePath) as src:
        out_image, out_transform = mask(src, geojson_geom, crop=True)
        out_meta = src.meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    with rasterio.open(outputPath, "w", **out_meta) as dest:
        dest.write(out_image)

def loopCutTifImage(tifFilePath, cutPointList, mainOutputPath):
    ''' 
    loop into every cut point 
    and check if the output file exist
    and cut the main tif img
    '''
    for pointCount, eachCutPoint in enumerate(cutPointList):
        # create this image path

        outputFolderKey = mainOutputPath.split('/')[-1] + "_" + str(pointCount+1)
        outputFolderName = os.path.join(mainOutputPath, outputFolderKey)
        os.makedirs(outputFolderName, exist_ok=True)

        outputFileName = outputFolderKey + "_original.tif"
        outputFilePath = os.path.join(outputFolderName, outputFileName)
        if os.path.exists(outputFilePath):
            print("File already exists: ", outputFilePath)
        else:
            cutPix4dRGBUsingCutPoints(tifFilePath, outputFilePath, eachCutPoint)
            print("saved new tif file to: ", outputFilePath)

    
def loopTifImage(tifFilePathList, cutPointList, mainOutputPath, imageType):
    ''' 
    loop into every tif image path 
    create RBG folder (if not exist)
    and call loop cut tif image inevery cut point
    '''
    for tifFilePath in tifFilePathList:
        # get file key for RGB path name
        fileKey = tifFilePath.split('/')[-1].split('_')[0]
        RBGFolderName = imageType + "_" + fileKey

        # create this image RGB path
        RGBFolderPath = os.path.join(mainOutputPath, RBGFolderName)
        os.makedirs(RGBFolderPath, exist_ok=True)

        loopCutTifImage(tifFilePath, cutPointList, RGBFolderPath)


In [7]:
tifFilePathRGB = "/Volumes/PortableSSD/2025Pix4DRGB"
tifFilePathDSM = "/Volumes/PortableSSD/2025Pix4DDSM"
cutPointPath = "./2025CutPoint.txt"
RGBOutPath = "/Volumes/PortableSSD/dataForProcess/2025MainData/RGB"
DSMOutPath = "/Volumes/PortableSSD/dataForProcess/2025MainData/DSM"

cutPointList = getCutPoint(cutPointPath)
tifFilePathList = getRGBTifFile(tifFilePathDSM)
loopTifImage(tifFilePathList, cutPointList, DSMOutPath, "DSM")

saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_1/DSM_202503010913_1_original.tif
saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_2/DSM_202503010913_2_original.tif
saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_3/DSM_202503010913_3_original.tif
saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_4/DSM_202503010913_4_original.tif
saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_5/DSM_202503010913_5_original.tif
saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_6/DSM_202503010913_6_original.tif
saved new tif file to:  /Volumes/PortableSSD/dataForProcess/2025MainData/DSM/DSM_202503010913/DSM_202503010913_7/DSM_202503010913_

In [8]:
# cutPix4dRGBUsingCutPoints(tifFilePathList[0], "./test.tif", cutPointList[0])